In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from service.Classifier.DataLoader.DataLoader import DataLoader
from service.Classifier.Enums.priority import Priority
from service.Classifier.DataLoader.P2DataLoader import P2DataLoader
from service.Classifier.DataLoader.P3DataLoader import P3DataLoader
from service.Classifier.DataLoader.P4DataLoader import P4DataLoader
from service.Classifier.Model.XGBRegressionModel import XGBRegressionModel
from service.Classifier.PreProcessing.RegressionModelPreProcessor import RegressionModelPreProcessor
from service.Classifier.DataSplit.TrainTestSplit import TrainTestSplit
from service.Classifier.Enums.dataTypeEnum import DataType

/Users/fjirigesi/Documents/OOSLAClassifier/service/Classifier/Model/XGBRegressionModel.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  from pandas import np


In [2]:
def initializeByPriority(priority):
    # load new updated data via Features class
    if priority == Priority.P2.value:
        return P2DataLoader()
    elif priority == Priority.P3.value:
        return P3DataLoader()
    elif priority == Priority.P4.value:
        return P4DataLoader()
    return None

In [3]:
priority = Priority.P2.value
def extract_data(priority):
    dataLoader = initializeByPriority(priority)
    data_df = dataLoader.loadTrainingRawData()
    # split data into train test 4:1
    dataSplit = TrainTestSplit()
    train_df, test_df = dataSplit.split(data_df)

    # generate features
    train_df, feature_names = dataLoader.transformRawDataToFeatures(train_df, DataType.TRAINDATA.value)
    test_df, feature_names = dataLoader.transformRawDataToFeatures(test_df, DataType.VALIDATION.value)
    # pre-processing the data based on model type
    preprocessor = RegressionModelPreProcessor(feature_names)
    train_X, train_y, test_X, test_y = preprocessor.preprocessing(train_df, test_df)
    
    return (train_X, train_y, test_X, test_y, train_df, test_df)

In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator

def evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df):

    model = xgb.XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.05).fit(train_X, train_y)
    predictions = np.rint(model.predict(test_X)).astype(np.int64)
    submission = pd.DataFrame({'ID': test_df['keyID'],
                               LabelEnum.CLOSEDDAY.value: predictions, 'Actual': test_y})
    rmse = np.sqrt(mean_squared_error(predictions, test_df[LabelEnum.CLOSEDDAY.value]))
    print("RMSE: %f" % rmse)
    mae = mean_absolute_error(predictions, test_y)
    print("MAE: %f" % mae)
    DataValidator().valid(submission, priority, test_df)
    print(confusion_matrix(submission["truth"], submission["prediction"]))
    print(classification_report(submission["truth"], submission["prediction"]))
    
    return (test_y, predictions, submission["truth"], submission["prediction"])

In [5]:
priority = Priority.P2.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p2_regression_true, p2_regression_pred, p2_classification_true, p2_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


RMSE: 8.873613
MAE: 6.066187
[[269 144]
 [ 39 243]]
              precision    recall  f1-score   support

       False       0.87      0.65      0.75       413
        True       0.63      0.86      0.73       282

    accuracy                           0.74       695
   macro avg       0.75      0.76      0.74       695
weighted avg       0.77      0.74      0.74       695



In [6]:
priority = Priority.P3.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p3_regression_true, p3_regression_pred, p3_classification_true, p3_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


RMSE: 19.314065
MAE: 11.903073
[[361 246]
 [ 52 187]]
              precision    recall  f1-score   support

       False       0.87      0.59      0.71       607
        True       0.43      0.78      0.56       239

    accuracy                           0.65       846
   macro avg       0.65      0.69      0.63       846
weighted avg       0.75      0.65      0.67       846



In [7]:
priority = Priority.P4.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p4_regression_true, p4_regression_pred, p4_classification_true, p4_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


RMSE: 21.416726
MAE: 14.185996
[[210 107]
 [ 44  96]]
              precision    recall  f1-score   support

       False       0.83      0.66      0.74       317
        True       0.47      0.69      0.56       140

    accuracy                           0.67       457
   macro avg       0.65      0.67      0.65       457
weighted avg       0.72      0.67      0.68       457



In [8]:
train_df

,keyID,priority,chainLength,longTimeHold,lateTriageDays,InitialActivityCount,InitialActionCount,recentActionCount,recentActivityCount,ResolvedDay,ClosedDay
0,COMS-8231,P4,3,0,0,1.363636,0.272727,0.045455,0.909091,65,65
1,CANCEL-6080,P4,1,0,0,0.545455,0.000000,0.000000,0.227273,68,68
2,COMS-7903,P4,0,0,0,0.772727,0.000000,0.090909,0.818182,46,46
3,CANCEL-6072,P4,0,0,0,0.363636,0.136364,0.181818,0.500000,61,61
4,BILLING-16551,P4,0,0,0,0.363636,0.090909,0.227273,0.500000,49,49
...,...,...,...,...,...,...,...,...,...,...,...
1928,VOD-3573,P4,0,0,0,0.227273,0.000000,0.045455,0.227273,68,68
1929,RES-21352,P4,1,0,9,0.454545,0.045455,0.045455,0.590909,50,57
1930,CANCEL-6268,P4,2,0,0,1.090909,0.136364,0.272727,0.727273,80,80
1931,RES-20850,P4,0,0,0,0.681818,0.090909,0.000000,0.090909,86,86


In [9]:
# # regression results
# regression_true = pd.concat([p2_regression_true, p3_regression_true])
# regression_true = pd.concat([regression_true, p4_regression_true])

# regression_pred = np.concatenate((p2_regression_pred, p3_regression_pred), axis=0)
# regression_pred = np.concatenate((regression_pred, p4_regression_pred), axis=0)

# ## classification results
# classification_true = pd.concat([p2_classification_true, p3_classification_true])
# classification_true = pd.concat([classification_true, p4_classification_true])

# classification_pred = pd.concat([p2_classification_pred, p3_classification_pred])
# classification_pred = pd.concat([classification_pred, p4_classification_pred])

In [10]:
# print("---Dynamic Features (including Activity Sequ) results on all data 75%---")
# rmse = np.sqrt(mean_squared_error(regression_pred, regression_true))
# print("RMSE: %f" % rmse)
# mae = mean_absolute_error(regression_pred, regression_true)
# print("MAE: %f" % mae)
# print(confusion_matrix(classification_true, classification_pred))
# print(classification_report(classification_true, classification_pred))